In [1]:
#  pip install google-cloud-bigquery -U
#  pip install pandas-gbq -U
import pandas as pd
from google.cloud import bigquery


In [4]:
# 3.280.108 repos -> 800 GB
# 1.000.000 repos -> 250 GB
# 100.000 repos -> 25 GB
# 10.000 repos -> 2.5 GB
client = bigquery.Client()

repos_num = 2 * 10 ** 4
query = f"""
  EXPORT DATA OPTIONS(
      uri='gs://github_activity_data/dataset/commits/*.json',
      format='json',
      overwrite=true) AS
  SELECT count(distinct(t.repo_name))
  FROM (
        SELECT repo_name 
        FROM github_activity_data.sample_repos
        LIMIT {repos_num}
                ) as t,
        github_activity_data.commits as com,
        UNNEST (com.repo_name) as rep

   WHERE rep in (t.repo_name);

  EXPORT DATA OPTIONS(
    uri='gs://github_activity_data/dataset/files/*.json',
    format='json',
    overwrite=true) AS
  SELECT fil.*
  FROM (
        SELECT repo_name 
        FROM github_activity_data.sample_repos
        LIMIT {repos_num}
        ) as t,
        github_activity_data.files as fil
  WHERE fil.repo_name = t.repo_name;

  EXPORT DATA OPTIONS(
      uri='gs://github_activity_data/dataset/languages/*.json',
      format='json',
      overwrite=true) AS
  SELECT com.*
  FROM (
        SELECT repo_name 
        FROM github_activity_data.sample_repos
        LIMIT {repos_num}
        ) as t,
        github_activity_data.languages as com

  WHERE com.repo_name = t.repo_name;

  EXPORT DATA OPTIONS(
      uri='gs://github_activity_data/dataset/licenses/*.json',
      format='json',
      overwrite=true) AS
  SELECT com.*
  FROM (
        SELECT repo_name 
        FROM github_activity_data.sample_repos
        LIMIT {repos_num}
        ) as t,
        github_activity_data.licenses as com

  WHERE com.repo_name = t.repo_name;
"""
print(repos_num)
print(query)


50000

  EXPORT DATA OPTIONS(
      uri='gs://github_activity_data/dataset/commits/*.json',
      format='json',
      overwrite=true) AS
  SELECT com.*
  FROM (
          SELECT repo_name 
          FROM github_activity_data.sample_repos
          LIMIT 50000
          ) as t,
          github_activity_data.commits as com

  WHERE com.repo_name[ordinal(1)] = t.repo_name;

  EXPORT DATA OPTIONS(
    uri='gs://github_activity_data/dataset/files/*.json',
    format='json',
    overwrite=true) AS
  SELECT fil.*
  FROM (
          SELECT repo_name 
          FROM github_activity_data.sample_repos
          LIMIT 50000
          ) as t,
          github_activity_data.files as fil
  WHERE fil.repo_name = t.repo_name;

  EXPORT DATA OPTIONS(
      uri='gs://github_activity_data/dataset/languages/*.json',
      format='json',
      overwrite=true) AS
  SELECT com.*
  FROM (
          SELECT repo_name 
          FROM github_activity_data.sample_repos
          LIMIT 50000
          ) as t,
    

In [5]:
query_job = client.query(query)  # API request
query_job.result()  # Waits for query to finish